<a href="https://colab.research.google.com/github/AnastasiaBlack/w20061036_Anastasiia_Ilina_Landmines_Detection_Research/blob/main/colabFiles/Two_Bounding_Boxes(GroundTruth_and_Detected).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code was used to create images that contain both the detected object rectangle with the ground truth rectangle drawn on the same image, but both rectangles have different colours.

In [ ]:
# convert normalized boxes to coordinates

def convert_normalized_to_coordinates(normalized_bbox):
  # Unpack the normalized bounding box
  x_center, y_center, width, height = normalized_bbox
  # Assuming image width and height
  image_width = 336
  image_height = 256
  # Convert normalized coordinates to image coordinates
  x_min = int((x_center - width / 2) * image_width)
  y_min = int((y_center - height / 2) * image_height)
  x_max = int((x_center + width / 2) * image_width)
  y_max = int((y_center + height / 2) * image_height)

  return x_min, y_min, x_max, y_max

In [ ]:
def load_boxes_from_txt(txt_path, is_normalized=True, images_with_no_detection=[], is_detection=False):
    try:
      with open(txt_path, 'r') as f:
        lines = f.readlines()
    except FileNotFoundError:
      print(f"The file {txt_path} does not exist.")
      if is_detection:
        images_with_no_detection.append(txt_path)
      return []

    boxes = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:
          confidence = float(parts[1])
          x_min, y_min, x_max, y_max = map(float, parts[1:])
        elif len(parts) == 4:
        # Assuming class_id is 0 if not provided
          x_min, y_min, x_max, y_max = map(float, parts[0:])
        elif len(parts) == 6:
          confidence = float(parts[1])
          x_min, y_min, x_max, y_max = map(float, parts[2:])

        if is_normalized:
            # Convert normalized coordinates to absolute coordinates
            x_min, y_min, x_max, y_max=convert_normalized_to_coordinates((x_min, y_min, x_max, y_max))

        if is_detection:
          boxes.append((float(confidence), int(x_min), int(y_min), int(x_max), int(y_max)))
        else:
          boxes.append((int(x_min), int(y_min), int(x_max), int(y_max)))

    return boxes

 The code is based on OpenCv library (https://docs.opencv.org/3.4/dc/da5/tutorial_py_drawing_functions.html)

In [ ]:
import cv2
import os
from PIL import Image
from xml.etree import ElementTree as ET

def draw_boxes(image_path, ground_truth_boxes, detected_boxes, output_path):
    # Read the image
    image = cv2.imread(image_path)

    # Draw ground truth bounding boxes in red
    for box in ground_truth_boxes:
        x_min, y_min, x_max, y_max = box
        # cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)  # Blue color
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)


    # Draw detected bounding boxes in green
    for box in detected_boxes:
        confidence, x_min, y_min, x_max, y_max = box
        print("Detected boxes: ", x_min, y_min, x_max, y_max)
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        # Prepare the label text
        label = f"'landmines' {confidence:.2f}%"  # Example format: "cat 82.00%"
        # Calculate the position to place the label above the bounding box
        # label_position = (x_min, y_min - 5)
        image_width=image.shape[1]
        image_height = image.shape[0]

        # Get the size of the label text
        (label_width, label_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)

        # Calculate the label position
        label_x = max(min(x_min, image_width - label_width), 0)
        label_y = max(min(y_min - 5, image_height - label_height), 0)
        label_position = (label_x, label_y)
        # Draw black border and green text
        cv2.putText(image, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)


    # Save or display the image
    cv2.imwrite(output_path, image)



In [ ]:
# Folder paths
import os
output_folder = "/content/result_boundingbox_images"
# Create the directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)


image_folder = "/content/drive/MyDrive/University/Research_Module/dataset/Yolov8_v3_dataset/Landmines detection dataset.v3i.yolov8/test/images"
ground_truth_folder = "/content/drive/MyDrive/University/Research_Module/dataset/Yolov8_v3_dataset/Landmines detection dataset.v3i.yolov8/test/labels"  # Change to your ground truth folder path
detected_folder = "/content/drive/MyDrive/University/Research_Module/development/Landmine detection/results_txt/detection_results_custom_YOLO"

images_with_no_detection=[]

# Iterate over images in the image folder
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)

    # Load ground truth and detected boxes
    ground_truth_path = os.path.join(ground_truth_folder, os.path.splitext(image_name)[0] + ".txt")
    detected_path = os.path.join(detected_folder, os.path.splitext(image_name)[0] + ".txt")

    ground_truth_boxes = load_boxes_from_txt(ground_truth_path, True)
    print("Initial ground_truth_boxes: ",ground_truth_boxes);
    detected_boxes = load_boxes_from_txt(detected_path, False, images_with_no_detection, True)
    print("Initial detected boxes: ",detected_boxes);

    # Draw and save the image with bounding boxes
    output_path = os.path.join(output_folder, os.path.splitext(image_name)[0] + "_result.jpg")
    draw_boxes(image_path, ground_truth_boxes, detected_boxes, output_path)
print("Images without detected objects: ", images_with_no_detection) #they do not include the cases when there are multiple objects and 1 was missed

Initial ground_truth_boxes:  [(81, 76, 155, 134)]
Initial detected boxes:  [(88.0, 83, 80, 149, 133)]
Detected boxes:  83 80 149 133
Initial ground_truth_boxes:  [(115, 71, 155, 102)]
Initial detected boxes:  [(82.0, 111, 72, 151, 103)]
Detected boxes:  111 72 151 103
Initial ground_truth_boxes:  [(105, 85, 198, 154)]
Initial detected boxes:  [(87.0, 107, 82, 198, 151)]
Detected boxes:  107 82 198 151
Initial ground_truth_boxes:  [(137, 100, 178, 139)]
Initial detected boxes:  [(62.0, 144, 110, 170, 135)]
Detected boxes:  144 110 170 135
Initial ground_truth_boxes:  [(113, 111, 137, 131)]
Initial detected boxes:  [(70.0, 112, 110, 138, 133)]
Detected boxes:  112 110 138 133
Initial ground_truth_boxes:  [(115, 126, 146, 150)]
Initial detected boxes:  [(75.0, 116, 122, 153, 151)]
Detected boxes:  116 122 153 151
Initial ground_truth_boxes:  [(113, 137, 143, 165)]
Initial detected boxes:  [(89.0, 110, 134, 144, 168)]
Detected boxes:  110 134 144 168
Initial ground_truth_boxes:  [(105, 91,

In [ ]:
# SSD run
# Folder paths
import os
output_folder = "/content/result_boundingbox_images_SSD"
# Create the directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)


image_folder = "/content/drive/MyDrive/University/Research_Module/dataset/Yolov8_v3_dataset/Landmines detection dataset.v3i.yolov8/test/images"
ground_truth_folder = "/content/drive/MyDrive/University/Research_Module/dataset/Yolov8_v3_dataset/Landmines detection dataset.v3i.yolov8/test/labels"  # Change to your ground truth folder path
detected_folder = "/content/drive/MyDrive/University/Research_Module/development/result_SSD_model/detection_result_txt"

images_with_no_detection=[]

# Iterate over images in the image folder
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)

    # Load ground truth and detected boxes
    ground_truth_path = os.path.join(ground_truth_folder, os.path.splitext(image_name)[0] + ".txt")
    detected_path = os.path.join(detected_folder, os.path.splitext(image_name)[0] + ".txt")

    ground_truth_boxes = load_boxes_from_txt(ground_truth_path, True)
    print("Initial ground_truth_boxes: ",ground_truth_boxes);
    detected_boxes = load_boxes_from_txt(detected_path, False, images_with_no_detection, True)
    print("Initial detected boxes: ",detected_boxes);

    # Draw and save the image with bounding boxes
    output_path = os.path.join(output_folder, os.path.splitext(image_name)[0] + "_result.jpg")
    draw_boxes(image_path, ground_truth_boxes, detected_boxes, output_path)
print("Images without detected objects: ", images_with_no_detection) #they do not include the cases when there are multiple objects and 1 was missed

Initial ground_truth_boxes:  [(81, 76, 155, 134)]
Initial detected boxes:  [(82.0, 82, 78, 149, 130)]
Detected boxes:  82 78 149 130
Initial ground_truth_boxes:  [(115, 71, 155, 102)]
Initial detected boxes:  [(112.0, 112, 73, 151, 102)]
Detected boxes:  112 73 151 102
Initial ground_truth_boxes:  [(105, 85, 198, 154)]
Initial detected boxes:  [(107.0, 107, 76, 198, 154)]
Detected boxes:  107 76 198 154
Initial ground_truth_boxes:  [(137, 100, 178, 139)]
The file /content/drive/MyDrive/University/Research_Module/development/result_SSD_model/detection_result_txt/91_Zone_7_Mine_5cm_depth__2-0m_TemperatureInCenter__32-C_jpg.rf.3b6a046ca434cbf82f1bc0d13b9b93ac.txt does not exist.
Initial detected boxes:  []
Initial ground_truth_boxes:  [(113, 111, 137, 131)]
Initial detected boxes:  [(187.0, 187, 49, 200, 60)]
Detected boxes:  187 49 200 60
Initial ground_truth_boxes:  [(115, 126, 146, 150)]
Initial detected boxes:  [(115.0, 115, 122, 148, 150)]
Detected boxes:  115 122 148 150
Initial gro

In [ ]:
# Save images with bounding boxes
# ZIP folder with all the content
import shutil

# Create a zip file for default test validation results with curves pngs:
shutil.make_archive('/content/result_boundingbox_images_SSD', 'zip', '/content/result_boundingbox_images_SSD')
from google.colab import files
files.download('/content/result_boundingbox_images_SSD.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>